**Authenticate to Kaggle**

In [13]:
mkdir %USERPROFILE%\.kaggle


In [ ]:
import shutil
import os

# Define source and destination paths
source = 'kaggle.json'
destination = os.path.join(os.path.expanduser("~"), ".kaggle", "kaggle.json")

# Copy the file
shutil.copy2(source, destination)


In [18]:
import os

kaggle_file = os.path.join(os.path.expanduser("~"), ".kaggle", "kaggle.json")

# Make the file read/write by the owner, and remove all other permissions.
os.chmod(kaggle_file, 0o600)


In [ ]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

In [29]:
import zipfile
zip_path = 'house-prices-advanced-regression-techniques.zip'
extract_folder = '.'  # Current directory

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

**Data Preprocessing**

In [1]:
import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
missing_values = df.isnull().sum()
missing_data = missing_values[missing_values > 0].sort_values()
missing_data

Electrical         1
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
BsmtExposure      38
BsmtFinType2      38
GarageCond        81
GarageQual        81
GarageFinish      81
GarageYrBlt       81
GarageType        81
LotFrontage      259
FireplaceQu      690
MasVnrType       872
Fence           1179
Alley           1369
MiscFeature     1406
PoolQC          1453
dtype: int64

In [67]:
missing_values

Id                 0
MSSubClass         0
MSZoning           0
LotFrontage      259
LotArea            0
                ... 
MoSold             0
YrSold             0
SaleType           0
SaleCondition      0
SalePrice          0
Length: 81, dtype: int64

In [39]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [3]:
abt = df.copy()

In [49]:
numerical_columns = df.select_dtypes(exclude='object').columns
categorical_columns = df.select_dtypes('object').columns

In [77]:
numerical_columns

Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd',
       'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF',
       'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
       'MiscVal', 'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

In [83]:
missing_data = df[numerical_columns].isnull().sum()
columns_with_missing_data = missing_data[missing_data > 0]

print(columns_with_missing_data)

LotFrontage    259
MasVnrArea       8
GarageYrBlt     81
dtype: int64


In [84]:
missing_data = df[categorical_columns].isnull().sum()
columns_with_missing_data = missing_data[missing_data > 0]

print(columns_with_missing_data)

Alley           1369
MasVnrType       872
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64


**Define a preprocessing funtion**

In [7]:
def preprocessing(df):
    # drop those columns with too many missing data
    df.drop(['Fence', 'Alley', 'MiscFeature', 'PoolQC', 'MasVnrType','FireplaceQu'], axis = 1, inplace = True)
    # filling missing columns (non-numerical)
    df['BsmtQual'].fillna('Missing', inplace = True)
    df['BsmtCond'].fillna('Missing', inplace = True)
    df['BsmtExposure'].fillna('Missing', inplace = True)
    df['BsmtFinType1'].fillna('Missing', inplace = True)
    df['BsmtFinType2'].fillna('Missing', inplace = True)
    df['Electrical'].fillna('Missing', inplace = True)
    df['GarageType'].fillna('Missing', inplace = True)
    df['GarageFinish'].fillna('Missing', inplace = True)
    df['GarageQual'].fillna('Missing', inplace = True)
    df['GarageCond'].fillna('Missing', inplace = True)


    # filling missing columns (numerical)
    df['LotFrontage'].fillna(df['LotFrontage'].mean(), inplace = True)
    df['MasVnrArea'].fillna(0, inplace = True)
    
    # Transforming 
    df['GarageAge'] = df['GarageYrBlt'].apply(lambda x: 2023.0 - x if not pd.isnull(x) else 0)
    df.drop('GarageYrBlt', axis = 1, inplace = True)


In [8]:
preprocessing(abt)

In [9]:
abt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 75 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1460 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [10]:
missing_values = abt.isnull().sum()
missing_data = missing_values[missing_values > 0].sort_values()
missing_data

Series([], dtype: int64)

**Model**

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [11]:
# creating features columns
X = abt.drop(['Id', 'SalePrice'], axis = 1)
# one hot encode
X = pd.get_dummies(X)
# creating target columns
y = abt['SalePrice']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)


In [13]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1022, 276), (438, 276), (1022,), (438,))

**Set up ML pipeline**

In [14]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [15]:
pipelines = {
    
    'rf': make_pipeline(StandardScaler(), RandomForestRegressor(random_state=1234)),
    'gb': make_pipeline(StandardScaler(), GradientBoostingRegressor(random_state=1234))
   

}

In [17]:
grid = {
    'rf': {
        'randomforestregressor__n_estimators':[100, 200, 300]

    },
    'gb': {
        'gradientboostingregressor__n_estimators':[100, 200, 300]
    }
}


In [18]:
fit_models = {}
# loop through all the algos
for algo, pipeline in pipelines.items():
  print(f'Training the {algo} model')
  # Create new Grid Search CV class
  model = GridSearchCV(pipeline, grid[algo], n_jobs = -1, cv=10)
  # train the model
  model.fit(X_train, y_train)
  # store it in our dictionary
  fit_models[algo] = model

Training the rf model
Training the gb model


In [19]:
fit_models

{'rf': GridSearchCV(cv=10,
              estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                        ('randomforestregressor',
                                         RandomForestRegressor(random_state=1234))]),
              n_jobs=-1,
              param_grid={'randomforestregressor__n_estimators': [100, 200,
                                                                  300]}),
 'gb': GridSearchCV(cv=10,
              estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                        ('gradientboostingregressor',
                                         GradientBoostingRegressor(random_state=1234))]),
              n_jobs=-1,
              param_grid={'gradientboostingregressor__n_estimators': [100, 200,
                                                                      300]})}

In [20]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [21]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    mae = mean_absolute_error(y_test, yhat)
    mse = mean_squared_error(y_test, yhat)
    rmse = mean_squared_error(y_test, yhat, squared=False)
    r2 = r2_score(y_test, yhat)

    print(algo)
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"R2: {r2}")

rf
MAE: 16461.55401826484
MSE: 675111622.9152334
RMSE: 25982.91020873592
R2: 0.8823530909900725
gb
MAE: 15202.421530445967
MSE: 594940493.4865786
RMSE: 24391.40204019807
R2: 0.8963239444741049


**Save Model**

In [22]:
import pickle

In [23]:
with open('gb.pkl','wb') as f:
    pickle.dump(fit_models['gb'], f)

In [24]:
with open('gb.pkl', 'rb') as f:
    reload_model = pickle.load(f)

**Test Data and make submission**

In [25]:
test_df = pd.read_csv('test.csv')

In [26]:
abt_test = test_df.copy()

In [27]:
preprocessing(abt_test)

In [28]:
# One hot encode
abt_test = pd.get_dummies(abt_test.drop('Id', axis = 1))

In [29]:
abt_test.shape

(1459, 259)

In [30]:
for col in X_train.columns:
    if col not in abt_test.columns:
        abt_test[col] = 0


In [31]:
abt_test = abt_test[X_train.columns]


In [32]:
missing_values = abt_test.isnull().sum()
missing_data = missing_values[missing_values > 0].sort_values()
missing_data

BsmtFinSF1      1
BsmtFinSF2      1
BsmtUnfSF       1
TotalBsmtSF     1
GarageCars      1
GarageArea      1
BsmtFullBath    2
BsmtHalfBath    2
dtype: int64

In [33]:
columns_to_fill = missing_data.index
abt_test[columns_to_fill] = abt_test[columns_to_fill].fillna(0)

In [35]:
missing_values = abt_test.isnull().sum()
missing_data = missing_values[missing_values > 0].sort_values()
missing_data

Series([], dtype: int64)

In [37]:
yhat_test = fit_models['gb'].predict(abt_test)
yhat

array([232363.08741234,  92672.97221549, 144698.48831952, 254592.48274626,
       138546.56118634, 219687.74865518, 332890.44007207, 123888.53139099,
       145344.92669809, 120386.91199644, 150484.662232  , 219828.16919304,
       139707.61392918, 105236.16486744, 257398.48404934, 191872.17447616,
       142484.73902003, 364090.85460085, 221783.1482415 , 187505.33125428,
       153390.19844792, 182361.27524494, 111536.87440254, 141184.06034031,
       213620.38213185, 150955.51206934, 216809.54344558, 187963.35621437,
       123403.72769693, 132794.10028725, 141503.9146153 , 215631.1089411 ,
       143152.45294837, 192747.2453021 , 359202.82910089, 199771.2142037 ,
       164324.90568778, 348139.55510554, 169481.76573367,  90439.1132724 ,
       144831.59348644, 195460.16744361, 207905.78434358, 169331.7300005 ,
       199518.27113726, 119737.63238991, 287435.69563755, 221621.52364467,
       243305.96547121, 119723.65146227, 212197.68089347, 184187.43981552,
       146271.43327975, 1

In [38]:
submission = pd.DataFrame([test_df['Id'],yhat_test]).T

In [39]:
submission.columns = ['Id', 'SalePrice']

In [42]:
submission['Id'] =submission['Id'].astype('float').astype('Int32')

In [43]:
submission.to_csv('kaggle_submission.csv', index=False)

In [44]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -m "initial gb model" -f "kaggle_submission.csv"

Successfully submitted to House Prices - Advanced Regression Techniques



  0%|          | 0.00/35.1k [00:00<?, ?B/s]
 46%|████▌     | 16.0k/35.1k [00:00<00:00, 22.4kB/s]
100%|██████████| 35.1k/35.1k [00:02<00:00, 13.3kB/s]
